In [ ]:
import httpx
from tqdm.autonotebook import tqdm
session = httpx.AsyncClient()

In [ ]:
r = await session.get('https://nominatim.pawin.me')

In [ ]:
r = await session.get("https://nominatim.pawin.me/search")
r.text

In [ ]:
async def lookup(place, excluded_nodes=[]):
  # print(excluded_nodes)
  excluded_place_ids = ",".join(map(str, excluded_nodes))
  # print(f"Excluded place ids: {excluded_place_ids}")
  url = f"https://nominatim.openstreetmap.org/search?format=json&q={place}&limit=50&exclude_place_ids={excluded_place_ids}&countrycodes=TH"
  print(f"URL Length: {len(url)}")
  print(url)
  response = await session.get(url)
  try:
    result = response.json()
  except ValueError:
    print(response.text)
    raise ValueError(f"Failed to decode JSON response: {response.text}")
  response.raise_for_status()
  return result

In [ ]:
def deduplicate_places(places):
  original_count = len(places)
  unique_ids = list(set(x['place_id'] for x in places))
  unique_places = [x for x in places if x['place_id'] in unique_ids]
  deduplicated_count = len(unique_places)
  duplicates = original_count - deduplicated_count
  return duplicates, unique_places

In [ ]:
[1, 2, 3] + [4, 5, 6]

In [ ]:
async def find_all_instances(place, limit_iters = 100):
  pbar = tqdm()
  allResult = []
  iter = 1
  duplicateCount = 0


  result = await lookup(place)
  allResult.extend(result)
  pbar.update(1)
  
  while (len(result) > 0) and (duplicateCount == 0) and (iter < limit_iters):
    excluded_nodes = [x['place_id'] for x in allResult]

    try:
      result = await lookup(place, excluded_nodes)
    except ValueError:
      print(f"Failed to lookup {place} with excluded nodes {excluded_nodes}")
      break
    duplicateCount, allResult = deduplicate_places(allResult + result)
    print(f"Got {len(result)} results, {duplicateCount} duplicates, {len(allResult)} total")

    pbar.update(1)
    iter += 1

  print(f"Found {len(allResult)} results, {duplicateCount} duplicates, {iter} iterations")
  print(result)

  return allResult
  

In [26]:
bangchak = deduplicate_places((await find_all_instances("บางจาก")) + (await find_all_instances('bangchak')))[1]

0it [00:00, ?it/s]

URL Length: 107
https://nominatim.openstreetmap.org/search?format=json&q=บางจาก&limit=50&exclude_place_ids=&countrycodes=TH
URL Length: 606
https://nominatim.openstreetmap.org/search?format=json&q=บางจาก&limit=50&exclude_place_ids=220932250,220697867,221749422,221590047,221374154,222032301,221273879,388618339,221555880,221309006,221287546,221355629,221850569,221369578,389836926,223612414,377893550,221872866,221301931,221548499,221604793,221227815,221376748,221583120,221874507,221342866,221494591,221272943,221592178,221399557,394350739,221563359,375501640,375344253,383989829,221701768,206368204,220154762,221728779,206323102,206673104,221388570,366302293,221566665,373684930,221636650,221416635,206575342,221482781,221444121&countrycodes=TH
Got 50 results, 0 duplicates, 100 total
URL Length: 1106
https://nominatim.openstreetmap.org/search?format=json&q=บางจาก&limit=50&exclude_place_ids=220932250,220697867,221749422,221590047,221374154,222032301,221273879,388618339,221555880,221309006,22128

0it [00:00, ?it/s]

URL Length: 109
https://nominatim.openstreetmap.org/search?format=json&q=bangchak&limit=50&exclude_place_ids=&countrycodes=TH
URL Length: 608
https://nominatim.openstreetmap.org/search?format=json&q=bangchak&limit=50&exclude_place_ids=221410435,221420495,221180174,221397682,221706601,221479113,381694553,221387200,206676640,206921421,220999671,206707779,220981550,221512224,221410695,221532410,221427607,206399420,206580709,221067398,221397984,206693518,206371053,206146150,220988489,221697851,221376748,221874537,221287546,221272943,223752795,221227815,221548499,221555880,389836926,223612414,221301931,221309006,221369578,388618339,221872866,221494591,221850569,394350739,221563359,375501640,375344253,383989829,221701768,206368204&countrycodes=TH
Got 50 results, 0 duplicates, 100 total
URL Length: 1108
https://nominatim.openstreetmap.org/search?format=json&q=bangchak&limit=50&exclude_place_ids=221410435,221420495,221180174,221397682,221706601,221479113,381694553,221387200,206676640,206921421

In [ ]:
ptt = deduplicate_places((await find_all_instances("ปตท")) + (await find_all_instances("ป.ต.ท")))[1]

In [30]:
pt = deduplicate_places((await find_all_instances("พีที")) + (await find_all_instances("pt")))[1]

0it [00:00, ?it/s]

URL Length: 105
https://nominatim.openstreetmap.org/search?format=json&q=พีที&limit=50&exclude_place_ids=&countrycodes=TH
URL Length: 154
https://nominatim.openstreetmap.org/search?format=json&q=พีที&limit=50&exclude_place_ids=221806228,207990334,208829047,208066214,207880412&countrycodes=TH
Got 50 results, 0 duplicates, 55 total
URL Length: 654
https://nominatim.openstreetmap.org/search?format=json&q=พีที&limit=50&exclude_place_ids=221806228,207990334,208829047,208066214,207880412,208915389,208748377,208178241,207738175,207822183,208292659,208498536,209660554,223497593,207739912,222622743,222813234,207027207,207568247,208219124,223212519,208817515,221369649,208654280,208776699,222870206,208980291,223383158,208876500,222342431,221950739,221645715,208252208,220292296,207215288,209318034,208359395,208320952,208776325,209013389,208723298,208978355,222490082,207358067,208639699,209262332,208575680,209521858,208041385,209545719,207024009,208457923,221966901,209060772,207677400&countrycodes=

0it [00:00, ?it/s]

URL Length: 103
https://nominatim.openstreetmap.org/search?format=json&q=pt&limit=50&exclude_place_ids=&countrycodes=TH
URL Length: 602
https://nominatim.openstreetmap.org/search?format=json&q=pt&limit=50&exclude_place_ids=222900428,224531885,224053608,372175429,223466545,208915389,208748377,209027280,376089439,224781167,209571670,222813234,208498536,223497593,208286813,209660554,208866129,394319129,378669046,208817515,208219124,393499463,394318619,223958400,394318620,222377480,223601115,220265286,222622743,221369649,208178241,223115877,208292659,207551157,221864328,221259639,222745567,223212519,223887746,207568247,207738175,207739912,222575948,221718605,393496556,222984739,207027207,207822183,223138020,208713365&countrycodes=TH
Got 50 results, 0 duplicates, 100 total
URL Length: 1102
https://nominatim.openstreetmap.org/search?format=json&q=pt&limit=50&exclude_place_ids=222900428,224531885,224053608,372175429,223466545,208915389,208748377,209027280,376089439,224781167,209571670,2228132

In [ ]:
shell = deduplicate_places((await find_all_instances("shell")) + (await find_all_instances("เชลล์")))[1]

In [31]:
caltex = deduplicate_places((await find_all_instances("caltex")) + (await find_all_instances("คาลเท็กซ์")))[1]

0it [00:00, ?it/s]

URL Length: 107
https://nominatim.openstreetmap.org/search?format=json&q=caltex&limit=50&exclude_place_ids=&countrycodes=TH
URL Length: 606
https://nominatim.openstreetmap.org/search?format=json&q=caltex&limit=50&exclude_place_ids=373929430,223801060,375559691,223018102,223836257,221583742,372561779,221520865,222317129,222973393,221561723,221721128,223081154,221637895,223555532,223084175,222715032,224299765,222507171,221834157,222840094,222195972,222250394,207675432,208925328,206861171,389121178,222620146,207245752,224448542,223319968,208566559,224204195,221990724,222524915,223395299,220311277,208772490,394338326,231343576,224009512,221561239,222892303,221786109,222929744,377474736,207172691,222351730,223037393,223626237&countrycodes=TH
Got 50 results, 0 duplicates, 100 total
URL Length: 1106
https://nominatim.openstreetmap.org/search?format=json&q=caltex&limit=50&exclude_place_ids=373929430,223801060,375559691,223018102,223836257,221583742,372561779,221520865,222317129,222973393,22156

0it [00:00, ?it/s]

URL Length: 110
https://nominatim.openstreetmap.org/search?format=json&q=คาลเท็กซ์&limit=50&exclude_place_ids=&countrycodes=TH
URL Length: 609
https://nominatim.openstreetmap.org/search?format=json&q=คาลเท็กซ์&limit=50&exclude_place_ids=223626237,221498563,224376368,209436505,224554865,223930379,223983851,221956763,208873977,209014744,221735535,223087019,222886835,223572849,222832309,208009134,208102959,223106536,222715032,223018102,223836257,209457459,222840094,222250394,208925328,207675432,206861171,223319968,207245752,208312399,215667898,223927614,207959987,222790445,207961786,207172691,215115984,209074702,224007375,223312336,207286799,206715599,208205284,208461195,215216321,223196630,222833979,221994046,222643665,223577808&countrycodes=TH
Got 50 results, 0 duplicates, 100 total
URL Length: 1109
https://nominatim.openstreetmap.org/search?format=json&q=คาลเท็กซ์&limit=50&exclude_place_ids=223626237,221498563,224376368,209436505,224554865,223930379,223983851,221956763,208873977,209014

In [ ]:
def ensure_petrol(places):
  return [place for place in places if place['type'] == "fuel"]

def not_pt(places):
  

In [32]:
mydata = {
  "bangchak": ensure_petrol(bangchak),
  "ptt": [place for place in ensure_petrol(ptt) if (place['name'].lower() != "p t") and (place['name'].lower() != "pt")],
  "pt": ensure_petrol(pt),
  "shell": ensure_petrol(shell),
  "caltex": ensure_petrol(caltex)
}

In [24]:
import json

In [33]:
json.dump(mydata, open("data.json", "w"), indent=2, ensure_ascii=False)